<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/2_week_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB0

In [2]:
vgg = VGG16(weights = 'imagenet')
resnet = ResNet50(weights = 'imagenet')
efficientnet = EfficientNetB0(weights = 'imagenet')

21848064/21834768 [==============================] - 0s 0us/step


In [3]:
print(vgg.summary())
"""
 [(None, 224, 224, 3)]
 Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
"""
print(resnet.summary())
"""
[(None, 224, 224, 3  0)]
Total params: 25,636,712
Trainable params: 25,583,592
Non-trainable params: 53,120
"""
print(efficientnet.summary())
"""
[(None, 224, 224, 3  0)]
Total params: 5,330,571
Trainable params: 5,288,548
Non-trainable params: 42,023
"""

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

'\n[(None, 224, 224, 3  0)]\nTotal params: 5,330,571\nTrainable params: 5,288,548\nNon-trainable params: 42,023\n'

In [4]:
# Denselayer 부분만 양자화 해주기
!pip install tensorflow_model_optimization
import tensorflow as tf
import tensorflow_model_optimization as tfmot

def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    vgg,
    clone_function=apply_quantization_to_dense,
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()


     |████████████████████████████████| 237 kB 15.5 MB/s 
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 bl

In [5]:
# 전체 양자화 해주기
quant_aware_model_all = tfmot.quantization.keras.quantize_model(vgg)
quant_aware_model_all.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 quantize_layer (QuantizeLay  (None, 224, 224, 3)      3         
 er)                                                             
                                                                 
 quant_block1_conv1 (Quantiz  (None, 224, 224, 64)     1923      
 eWrapperV2)                                                     
                                                                 
 quant_block1_conv2 (Quantiz  (None, 224, 224, 64)     37059     
 eWrapperV2)                                                     
                                                                 
 quant_block1_pool (Quantize  (None, 112, 112, 64)     1         
 WrapperV2)                                                  

In [9]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!mkdir tiny_imagenet

--2022-03-21 12:41:46--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip.2’

tiny-imagenet-200.z 100%[===================>] 236.61M  39.7MB/s    in 8.4s    

2022-03-21 12:41:54 (28.0 MB/s) - ‘tiny-imagenet-200.zip.2’ saved [248100043/248100043]



In [10]:
import glob
import os
import shutil

src_path = "./tiny_imagenet/"
train_dst_path = "./train/"
test_dst_path = "./test/"
split_ratio = 0.9



def check_folder_and_make(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def main():
    check_folder_and_make(train_dst_path)
    check_folder_and_make(test_dst_path)
    folders = os.listdir(src_path)
    for folder in folders:
        check_folder_and_make(train_dst_path+folder)
        check_folder_and_make(test_dst_path+folder)
        img_paths = glob.glob(src_path+folder+"/images/*.JPEG")

        img_len = len(img_paths)
        train_index = int(img_len * split_ratio)

        train_set = img_paths[:train_index]
        test_set = img_paths[train_index:]

        for train in train_set:
            shutil.copy2(train, train_dst_path+folder)

        for test in test_set:
            shutil.copy2(test, test_dst_path+folder)

if __name__ == "__main__":
    main()